In [ ]:
# # This file is part of Theano Geometry
#
# Copyright (C) 2017, Stefan Sommer (sommer@di.ku.dk)
# https://bitbucket.org/stefansommer/theanogemetry
#
# Theano Geometry is free software: you can redistribute it and/or modify
# it under the terms of the GNU General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.
#
# Theano Geometry is distributed in the hope that it will be useful,
# but WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
# GNU General Public License for more details.
#
# You should have received a copy of the GNU General Public License
# along with Theano Geometry. If not, see <http://www.gnu.org/licenses/>.
#

# Frame Bundle Geometry on Embedded Ellipsoids

In [ ]:
from src.manifolds.ellipsoid import *
M = Ellipsoid(params=np.array([1,1,1]))
#M.params.set_value(np.array([1,0.5,0.5]))
print(M)
from src.plotting import *

In [ ]:
# Riemannian structure
from src.Riemannian import metric
metric.initialize(M,truncate_high_order_derivatives=False)

# geodesics
from src.Riemannian import geodesic
geodesic.initialize(M)

In [ ]:
# frame bundle
from src.framebundle import FM
FM.initialize(M)

In [ ]:
# test that adapated bases D and D^* are dual
x1 = M.coordsf([1.2,0.])
nu1 = np.dot(np.diag((.5,1.)),np.linalg.cholesky(M.gsharpf(x1)))
u1 = (np.concatenate((x1[0],nu1.flatten())),x1[1])

A = np.zeros((6,6))
for i in range(6):
    Dp1 = np.zeros(6)
    Dp1[i] = 1.
    p1 = M.from_Dstarf(u1,Dp1)
    
    for j in range(6):
        Dv1 = np.zeros(6)
        Dv1[j] = 1.
        v1 = M.from_Df(u1,Dv1)

        A[i,j] = np.dot(v1,p1)
print(A)

A = np.zeros((6,6))
for i in range(6):
    p1 = np.zeros(6)
    p1[i] = 1.
    Dp1 = M.to_Dstarf(u1,p1)
    
    for j in range(6):
        v1 = np.zeros(6)
        v1[j] = 1.
        Dv1 = M.to_Df(u1,v1)

        A[i,j] = np.dot(Dv1,Dp1)
print(A)

In [ ]:
# elements
x = M.coordsf([0.,0.])

# element u=(x,nu) in FM, nu being frame for T_xM
# np.linalg.cholesky(M.gsharpf(x)) gives orthonormal basis for T_xM, multiplication scales in given directions
nu = np.dot(np.diag((.5,1.)),np.linalg.cholesky(M.gsharpf(x)))
u = (np.concatenate((x[0],nu.flatten())),x[1])

# FM covector p
v = tensor([2.,2.])
px = np.linalg.solve(nu,v) # manifold part
pu = tensor([0.,0.,0.,0.]) # frame part
p = np.concatenate([px,pu])

print("u = ", u)
print("p = ", p)

newfig()
M.plot()
M.plotx(x,v=nu)

## FM Geodesics

In [ ]:
# Hamiltionian dynamics on FM from sub-Riemannian structure <v,w>_FM=<u^-1(v),u^-1(w)>_R^2
from src.framebundle import Hamiltonian_FM
Hamiltonian_FM.initialize(M)

In [ ]:
# test that chart update preserves FM Hamiltonian
x1 = M.coordsf([1.2,0.])
nu1 = np.dot(np.diag((.5,1.)),np.linalg.cholesky(M.gsharpf(x1)))
u1 = (np.concatenate((x1[0],nu1.flatten())),x1[1])

for i in range(6):
#     Dp1 = np.zeros(6)
#     Dp1[i] = 1.
#     p1 = M.from_Dstarf(u1,Dp1)
    p1 = np.zeros(6)
    p1[i] = 1.

    print("M.H_FM x1:",M.H_FMf(u1,p1))
    chart2 = M.centered_chartf(M.Ff(x1))
    up2 = M.chart_update_Hamiltonian_FMf(u1,p1)
    print("M.H_FM x2:",M.H_FMf((up2[0],chart2),up2[1]))

In [ ]:
print(M.H_FMf(u,p))

# compute FM geodesic
(us,charts) = M.Exp_Hamiltonian_FMtf(u,p)

# plot
newfig()
M.plot(rotate=(30,80))
M.plot_path(zip(us,charts),v_steps=np.arange(0,n_steps.eval(),5),linewidth=1.5,s=50)
plt.show()

# dynamics returning both position and momentum
(ts,qps,charts) = M.Hamiltonian_dynamics_FMf(u,p)
us = qps[:,0,:]
ps = qps[:,1,:]
print("Energy: ",np.array([M.H_FMf((q,charts),p) for (q,p,charts) in zip(us,ps,charts)]))

# Development and Stochastic Development

In [ ]:
# development dynamics
from src.stochastics import stochastic_development
stochastic_development.initialize(M)

In [ ]:
# deterministic development

# curve in R^2
t = np.linspace(0,10,n_steps.get_value()+1)
gamma = np.vstack([[20*np.sin(t), t**2 + 2*t]]).T
dgamma = np.diff(gamma, axis = 0)

(ts,us,charts) = M.developmentf(u,dgamma)

# plot with frame
newfig()
M.plot()
M.plot_path(zip(us,charts),v_steps=np.arange(0,n_steps.eval(),5))
plt.show()

# plot only trajectory
newfig()
M.plot()
M.plot_path(zip(us[:,0:M.dim.eval()],charts))
plt.show()

# plot anti-development
plt.figure()
plt.plot(gamma[:,0],gamma[:,1])
plt.axis('equal')
plt.show()

In [ ]:
n_steps.set_value(1000)

# stochastic development
w = dWsf(M.dim.eval()) # noise / anti-development
(ts,us,charts) = M.stochastic_developmentf(u,w)

# plot with frame
newfig()
M.plot()
M.plot_path(zip(us,charts),v_steps=np.arange(0,n_steps.eval(),50))
plt.show()

# plot only trajectory
newfig()
M.plot()
M.plot_path(zip(us[:,0:M.dim.eval()],charts))
plt.show()

# plot noise / anti-development
plt.figure()
ws = np.cumsum(w,axis=0)
plt.plot(ws[:,0],ws[:,1])
plt.axis('equal')
plt.show()

n_steps.set_value(100)

In [ ]:
# # Delyon/Hu guided process
from src.stochastics.guided_process import *

# guide function
Cholesky = T.slinalg.Cholesky()
def phi(u,v):
    x = (u[0][0:M.dim],u[1])
    nu = u[0][M.dim:].reshape((M.dim,-1))
    
    return T.nlinalg.tensorsolve(nu,M.StdLog(x,v).flatten()).reshape((M.dim,))

(stochastic_development_guided,stochastic_development_guidedf) = get_guided_likelihood(
    M,M.sde_development,phi,lambda x: T.eye(M.dim),A=lambda x,v,w: T.dot(v,w),
    use_charts=True,chart_update=M.chart_update_FM)

n_steps.set_value(1000)

w = M.Ff(M.Expf(x,tensor(np.array([.8,-.5]))))
(ts,us,charts,log_likelihood,log_varphi) = stochastic_development_guidedf(u,w,dWsf(M.dim.eval()))[:5]
print("log likelihood: ", log_likelihood[-1], ", log varphi: ", log_varphi[-1])

# plot
newfig()
M.plot()
M.plot_path(zip(us,charts),v_steps=np.arange(0,n_steps.eval(),50))
M.plotx(x,color='r',s=150)
M.plotx(w,color='k',s=150)
plt.show()

n_steps.set_value(100)

# Anisotropic  Normal Distribution

In [ ]:
# plot sample data with trajectories
K = 8
obss = np.zeros((K,n_steps.eval(),M.dim.eval()))
obs_charts = np.zeros((K,n_steps.eval(),)+x[1].shape)
# srng.seed(422)
i = 0
while i < K:
    try:
        (ts,us,charts) = M.stochastic_developmentf(u,dWsf(M.dim.eval()))
        obss[i] = us[:,0:M.dim.eval()]
        obs_charts[i] = charts
        i += 1
    except np.linalg.linalg.LinAlgError:
        pass

# plot samples
colormap = plt.get_cmap('winter')
colors=[colormap(k) for k in np.linspace(0, 1, K)]
newfig()
M.plot()
M.plotx(x,v=u[0][M.dim.eval():].reshape((M.dim.eval(),-1)))
for i in range(K):
    M.plot_path(zip(obss[i],obs_charts[i]),linewidth=.5,color=colors[i])
plt.show()

In [ ]:
# sample data
K = 1024
obss = np.zeros((K,M.dim.eval()))
obs_charts = np.zeros((K,)+x[1].shape)
# srng.seed(422)
i = 0
while i < K:
    try:
        (ts,us,charts) = M.stochastic_developmentf(u,dWsf(M.dim.eval()))
        obss[i] = us[-1][0:M.dim.eval()]
        obs_charts[i] = charts[-1]
        i += 1
    except np.linalg.linalg.LinAlgError:
        pass

# plot samples
newfig()
M.plot()
M.plotx(x,v=u[0][M.dim.eval():].reshape((M.dim.eval(),-1)))
for i in range(K):
    M.plotx((obss[i],obs_charts[i]))
plt.show()

In [ ]:
# # plot estimated density, 
# newfig()
# # plotM(alpha=.4)
# # plot_sphere_density_estimate(M, np.array([M.Ff(obs) for obs in obss]),pts=100,alpha=.8,bandwidth=.15) # spherical coordinates
# plot_density_estimate(M,np.array([M.Ff((obs,chart)) for (obs,chart) in zip(obss,obs_charts)]),limits=[-3,3,-3,3],pts=500,alpha=.4,bandwidth=.15) # general ellipsoidal coordinates (note: very long computation time)
# plt.show()

# Most Probable Paths

In [ ]:
def initialize(M):
    y = M.sym_element()
    y_chart = M.sym_chart()
    p = M.sym_covector()
    
    def loss(u,p,y):
        d = y[0].shape[0]
        (u1,chart1) = M.Exp_Hamiltonian_FM(u,p)
        y_chart1 = M.update_coords(y,chart1)
        return 1./d*T.sum(T.sqr(u1[0:d] - y_chart1[0]))
    M.lossf = M.coords_function(lambda u,p,y,y_chart: loss(u,p,(y,y_chart)),p,y,y_chart)

    def Log_FM(u,y):
        res = minimize(lambda p: M.lossf(u,p,y[0],y[1]), np.zeros(u[0].shape), 
                       method='CG', jac=False, options={'disp': False, 
                                                        'maxiter': 50})
        return res.x
    M.Log_FM = Log_FM
initialize(M)

In [ ]:
# Compute 'most probable path' (in the sense of the driving semi-martingale) between u and x2
x2 = M.coordsf([0.25,1.])

# cotangent vector for the MPP:
px2 = M.Log_FM(u,x2)

# MPP from u to x2:
(us,charts) = M.Exp_Hamiltonian_FMtf(u,px2)

# plot
newfig()
M.plot(rotate=(30,80))
M.plotx(x,color="blue")
M.plotx(x2,color="red")
M.plot_path(zip(us,charts),v_steps=np.arange(0,n_steps.eval(),5),linewidth=1.5,s=50)
plt.show()

# Horizontal Vector Fields

In [ ]:
def plotHorizontal(u,color='b',color_intensity=1.,linewidth=3.,prevx=None,last=True):
        chart = u[1]    
        x = (u[0][0:M.dim.eval()],chart)
        nu = u[0][M.dim.eval():].reshape((M.dim.eval(),-1))
        xM = M.Ff(x)
        
        ax = plt.gca()
        
        # plot frame and horizontal variation
        M.plotx(x)
        Hu = M.Horizontalf(u) # horizontal basis fields
        print("Hu:",Hu)
        Hnu = Hu[M.dim.eval():].reshape((M.dim.eval(),nu.shape[1],nu.shape[1])) # nu part
        JFx = M.JFf(x)
        for j in range(M.dim.eval()):
            nujM = np.dot(JFx,nu[:,j])
            HnujM = np.dot(JFx,np.dot(Hnu,nu[:,j]))
            ax.quiver(xM[0],xM[1],xM[2],nujM[0],nujM[1],nujM[2], pivot='tail',
                      arrow_length_ratio = 0.15, linewidths=1,
                      color='black',normalize=True,length=np.linalg.norm(nujM))
            for k in range(nu.shape[1]):
                basep = xM + nujM
                ax.quiver(basep[0],basep[1],basep[2],
                          HnujM[0,k],HnujM[1,k],HnujM[2,k], pivot='tail',linewidths=2.,
                          color='red',normalize=True,length=0.3)


# plot horizontal vector fields
newfig()
M.plot()
plotHorizontal(u)
plt.show()